In [15]:
import numpy as np
from tqdm import tqdm
import time
from numba import njit, prange, jit
from typing import NamedTuple
import zlib
from collections import deque
from itertools import product

In [16]:
# Create a named Tuple class to store the problems
class ProblemInstance(NamedTuple):
    holding_costs: np.ndarray
    setup_costs: np.ndarray
    setup_times: np.ndarray
    demand: np.ndarray
    production_times: np.ndarray
    capacities: np.ndarray
    num_periods: int
    num_products: int

# Create a class to store the population
class Population(NamedTuple):
    population: np.ndarray 
    fitness: np.ndarray 

In [17]:
dem = np.array([[0, 0, 0, 39, 26, 50, 38, 0],
                [0, 0, 3, 0, 45, 42, 10, 29],
                [0, 50, 30, 0, 23, 0, 0, 20],
                [0, 37, 0, 40, 0, 36, 0, 0],
                [200, 39, 39, 10, 0, 43, 30, 0]],dtype=np.int32)
demand = []

for i in range(5):

    for x in range(5):
        demand.append([0 for i in range(8)])
    demand.append(dem[i])

demand = np.array(demand,dtype=np.int32)
problem_excel_new = ProblemInstance(
    holding_costs = np.array([1,1,1,1,1,1,1,1],dtype=np.int32),
    setup_costs = np.array([[0, 152, 139, 143, 196, 113, 210, 110],
                            [152, 0, 105, 195, 196, 127, 150, 175],
                            [139, 105, 0, 108, 198, 171, 125, 125],
                            [143, 195, 108, 0, 117, 157, 120, 176],
                            [196, 196, 198, 117, 0, 162, 220, 177],
                            [113, 127, 171, 157, 162, 0, 120, 130],
                            [210, 150, 125, 120, 220, 120, 0, 200],
                            [110, 175, 125, 176, 177, 130, 200, 0]],dtype=np.int32),
setup_times = np.array([[0, 11, 18, 18, 11, 13, 15, 18],
                        [11, 0, 15, 13, 17, 14, 16, 11],
                        [18, 15, 0, 16, 12, 12, 15, 11],
                        [18, 13, 16, 0, 18, 16, 10, 10],
                        [11, 17, 12, 18, 0, 20, 20, 19],
                        [13, 14, 12, 16, 20, 0, 15, 12],
                        [15, 16, 15, 10, 20, 15, 0, 12],
                        [18, 11, 11, 10, 19, 12, 12, 0]],dtype=np.int32),
 demand = demand,

 production_times = np.array([1,1,1,1,1,1,1,1]),
 
 capacities = np.array([38 for i in range(30)],dtype=np.int32),
        num_periods = 30,
        num_products = 8)

In [18]:
dem = np.array([[0, 0, 0, 39, 26, 50, 38, 0],
                [0, 0, 3, 0, 45, 42, 10, 29],
                [0, 50, 30, 0, 23, 0, 0, 20],
                [0, 37, 0, 40, 0, 36, 0, 0],
                [0, 39, 39, 10, 0, 43, 30, 0],
                [0, 0, 0, 39, 26, 50, 38, 0],
                [0, 0, 3, 0, 45, 42, 10, 29],
                [0, 50, 30, 0, 23, 0, 0, 20],
                [0, 37, 0, 40, 0, 36, 0, 0],
                [400, 39, 39, 10, 0, 43, 30, 0]],dtype=np.int32)
demand = []

for i in range(10):

    for x in range(5):
        demand.append([0 for i in range(8)])
    demand.append(dem[i])

demand = np.array(demand,dtype=np.int32)
problem_excel_x2 = ProblemInstance(
    holding_costs = np.array([1,1,1,1,1,1,1,1],dtype=np.int32),
    setup_costs = np.array([[0, 152, 139, 143, 196, 113, 210, 110],
                            [152, 0, 105, 195, 196, 127, 150, 175],
                            [139, 105, 0, 108, 198, 171, 125, 125],
                            [143, 195, 108, 0, 117, 157, 120, 176],
                            [196, 196, 198, 117, 0, 162, 220, 177],
                            [113, 127, 171, 157, 162, 0, 120, 130],
                            [210, 150, 125, 120, 220, 120, 0, 200],
                            [110, 175, 125, 176, 177, 130, 200, 0]],dtype=np.int32),
setup_times = np.array([[0, 11, 18, 18, 11, 13, 15, 18],
                        [11, 0, 15, 13, 17, 14, 16, 11],
                        [18, 15, 0, 16, 12, 12, 15, 11],
                        [18, 13, 16, 0, 18, 16, 10, 10],
                        [11, 17, 12, 18, 0, 20, 20, 19],
                        [13, 14, 12, 16, 20, 0, 15, 12],
                        [15, 16, 15, 10, 20, 15, 0, 12],
                        [18, 11, 11, 10, 19, 12, 12, 0]],dtype=np.int32),
 demand = demand,

 production_times = np.array([1,1,1,1,1,1,1,1]),
 
 capacities = np.array([38 for i in range(60)],dtype=np.int32),
        num_periods = 60,
        num_products = 8)
# Current best: 12503

In [27]:
# ========== DECODE AND EVALUATE FUNTIONS ==========

def decode_verbose(demand, capacities, production_times, setup_times,
           solution_full, holding_costs, setup_costs, penalty_shortage=1):

    # ========== BACKLOG MATRIX ==========

    periods, products = demand.shape
    Skip_production = products
    setup_change = products + 1
    setup_change_direct = products + 2
    solution = solution_full[:, :-3]

    # Create the backlog matrix and a matrix for the cumulative demand
    backlog = np.zeros((periods, products), dtype=np.int32)
    cum_demand = np.zeros(products, dtype=np.int32)

    # For each period we add the demand to the cum_demand array which subsequently gets added to the backlog matrix
    # The backlog matrix shows (beginning with the last period) how many products still need to be produced
    for per in range(periods):
        src = periods - per - 1
        for prod in range(products):
            cum_demand[prod] += demand[src, prod]
        for prod in range(products):
            backlog[src, prod] = cum_demand[prod]

    # ========== LAST PERIOD ==========
            
    # ========== LAST PERIOD ==========
            
    # Create a matrix for the decoded solution
    decoded = np.zeros((periods + 1, 3), dtype=np.int32)
    

    # Get the index of the last period
    period = periods - 1

    # Get the valid products (those with backlog > 0)
    valid_products = np.full(products, -1.0)


    # If there is demand in the previous period, but not in this one, we can change the setup
    if np.all(backlog[period] == 0):
        for p in range(products):
            valid_products[p] = solution[period, p]
    else:
        for p in range(products):
            if backlog[period, p] > 0:
                valid_products[p] = solution[period, p]

    # Select the product with the highest priority
    second_product = np.argmax(valid_products)
    decoded[period+1, 2] = second_product


    capacity = capacities[period]

    # If the binary indicator for a setup change is 1, directly plan the setup change so that we do not violate capacity restrictions
    if solution_full[period, setup_change] == 1:

        # Valid products this time also include those from the next period
        back = backlog[period] + backlog[period - 1]
        valid_products_2 = np.full(products, -1.0)
        for p in range(products):
            if back[p] > 0:
                valid_products_2[p] = solution[period, p]

        # We set the value of the first product to -1, so that it cannot be selected
        valid_products_2[second_product] = -1
        first_product = np.argmax(valid_products_2)

        # Set the first product and subtract the setup times from the capcity
        decoded[period, 2] = first_product
        capacity -= setup_times[first_product, second_product]

        # Produce the second product either until the demand or capacity is satisfied
        max_quantity = capacity // production_times[second_product]
        quantity = min(max_quantity, backlog[period, second_product])
        decoded[period + 1, 0] = quantity
        for j in range(periods):
            backlog[j, second_product] -= quantity

        capacity -= quantity * production_times[second_product]

        # Produce the first product if there is spare capacity
        if capacity > 0 and solution_full[period, Skip_production] == 0:
            max_quantity = capacity // production_times[first_product]
            quantity = min(max_quantity, backlog[period, first_product])
            decoded[period, 1] = quantity
            for j in range(periods):
                backlog[j, first_product] -= quantity

    # If we do not change the setup, just produce the second product 
    else:
        max_quantity = capacity // production_times[second_product]
        quantity = min(max_quantity, backlog[period, second_product])
        decoded[period + 1, 0] = quantity
        for j in range(periods):
            backlog[j, second_product] -= quantity

        decoded[period, 2] = second_product

    # ========== OTHER PERIODS ==========
            
    for i in range(len(solution) - 1):

        # Start with the second to last period and work to the beginning
        period = periods - 2 - i

        # Get the valid products (those with backlog > 0)
        valid_products = np.full(products, -1.0)
        # If there is no demand in this or the previous period, we do not need to produce or change the setup
        if np.all(backlog[period] + backlog[period-1] == 0):
            decoded[period,2] = decoded[period+1,2]
            continue


        # If there is demand in the previous period, but not in this one, we can change the setup
        elif np.all(backlog[period] == 0):
            for p in range(products):
                valid_products[p] = solution[period, p]
        else:
            for p in range(products):
                if backlog[period, p] > 0:
                    valid_products[p] = solution[period, p]

        # Select the product with the highest priority
        second_product = np.argmax(valid_products)

        # If the product with the highest priority is the same as in the next period, we do not need to change the setup
        if second_product == decoded[period + 1, 2]:
            capacity = capacities[period]

            # If the binary indicator for a setup change is 1, directly plan the setup change so that we do not violate capacity restrictions
            if solution_full[period, setup_change] == 1:

                # Valid products this time also include those from the next period
                back = backlog[period] + backlog[period - 1]
                valid_products_2 = np.full(products, -1.0)
                for p in range(products):
                    if back[p] > 0:
                        valid_products_2[p] = solution[period, p]

                # We set the value of the first product to -1, so that it cannot be selected
                valid_products_2[second_product] = -1
                first_product = np.argmax(valid_products_2)

                # Set the first product and subtract the setup times from the capcity
                decoded[period, 2] = first_product
                capacity -= setup_times[first_product, second_product]

                # Produce the second product either until the demand or capacity is satisfied
                max_quantity = capacity // production_times[second_product]
                quantity = min(max_quantity, backlog[period, second_product])
                decoded[period + 1, 0] = quantity
                for j in range(periods):
                    backlog[j, second_product] -= quantity

                capacity -= quantity * production_times[second_product]

                # Produce the first product if there is spare capacity
                if capacity > 0 and solution_full[period, Skip_production] == 0:
                    max_quantity = capacity // production_times[first_product]
                    quantity = min(max_quantity, backlog[period, first_product])
                    decoded[period, 1] = quantity
                    for j in range(periods):
                        backlog[j, first_product] -= quantity

            # If we do not change the setup, just produce the second product 
            else:
                max_quantity = capacity // production_times[second_product]
                quantity = min(max_quantity, backlog[period, second_product])
                decoded[period + 1, 0] = quantity
                for j in range(periods):
                    backlog[j, second_product] -= quantity

                decoded[period, 2] = second_product

        elif solution_full[period, setup_change_direct] == 1:

            if backlog[period, decoded[period+1, 2]] > 0:
                second_product = decoded[period+1, 2]
                decoded[period, 2] = second_product
                spare_capacity = capacities[period] - setup_times[second_product, decoded[period + 1, 2]]
                max_quantity = spare_capacity // production_times[second_product]
                quantity = min(max_quantity, backlog[period, second_product])
                decoded[period, 1] = quantity
                for j in range(periods):
                    backlog[j, second_product] -= quantity

            elif solution_full[period, Skip_production] == 0:
                decoded[period, 2] = second_product
                spare_capacity = capacities[period] - setup_times[second_product, decoded[period + 1, 2]]
                max_quantity = spare_capacity // production_times[second_product]
                quantity = min(max_quantity, backlog[period, second_product])
                decoded[period, 1] = quantity
                for j in range(periods):
                    backlog[j, second_product] -= quantity

            else:
                decoded[period, 2] = decoded[period + 1, 2]


        # If the product with the highest priority differs from the one in the last period, change the setup directly
        elif solution_full[period, Skip_production] == 0:
            decoded[period, 2] = second_product
            spare_capacity = capacities[period] - setup_times[second_product, decoded[period + 1, 2]]
            max_quantity = spare_capacity // production_times[second_product]
            quantity = min(max_quantity, backlog[period, second_product])
            decoded[period, 1] = quantity
            for j in range(periods):
                backlog[j, second_product] -= quantity

        # If we skip production, just transfer the setup from the last period
        else:
            decoded[period, 2] = decoded[period + 1, 2]

    periods, products = demand.shape

    lots1 = decoded[1:, 0].astype(np.int32)
    lots2 = decoded[:-1, 1].astype(np.int32)
    seq = decoded[:, 2].astype(np.int32)

    # --- Production matrix ---
    production = np.zeros((periods, products), dtype=np.int32)
    for t in range(periods):
        production[t, seq[t + 1]] += lots1[t]
        production[t, seq[t]] += lots2[t]

    print("\n=== Production Matrix ===")
    print(production)

    # --- Inventory matrix ---
    inventory = np.zeros((periods, products), dtype=np.float64)
    for m in range(products):
        cum = 0.0
        for t in range(periods):
            cum += production[t, m] - demand[t, m]
            inventory[t, m] = cum

    print("\n=== Inventory Matrix ===")
    print(inventory)

    # --- Holding and shortage costs ---
    hold_costs_per_product = np.sum(inventory * holding_costs, axis=0)
    hold_cost_total = np.sum(hold_costs_per_product)

    shortage_violations = inventory < 0
    shortage_units = -np.sum(inventory[shortage_violations])
    shortage_penalty = shortage_units * penalty_shortage

    print("\n--- Holding Costs per Product ---")
    for i in range(products):
        print(f"Product {i}: {hold_costs_per_product[i]:.2f}")

    print("\nTotal Holding Cost:", hold_cost_total)
    print("Total Shortage Penalty:", shortage_penalty)
    print("Total Shortage Violations (units):", shortage_units)

    # --- Setup and production time ---
    setup_time = np.zeros(periods, dtype=np.int32)
    for t in range(periods):
        setup_time[t] = setup_times[seq[t], seq[t + 1]]

    prod_time = np.sum(production * production_times, axis=1)
    total_time = prod_time + setup_time
    overtime_violations = np.maximum(total_time - capacities, 0)

    print("\nSetup Times per Period:", setup_time)
    print("Production Times per Period:", prod_time)
    print("Total Time per Period:", total_time)
    print("Overtime Violations (units):", np.sum(overtime_violations))

    # --- Setup cost ---
    setup_c = 0.0
    for t in range(periods):
        setup_c += setup_costs[seq[t], seq[t + 1]]

    print("\nTotal Setup Cost:", setup_c)

    total_cost = hold_cost_total + shortage_penalty + setup_c
    print("\n=== Final Total Cost ===")
    if shortage_penalty == 0:
        print(f'\nFeasible Solution found with Total Cost: {total_cost}')
    else:
        print(f'\nNo feasible Solution found')

    return decoded

@njit
def decode_num(demand, capacities, production_times, setup_times,
           solution_full, holding_costs, setup_costs, penalty_shortage=1):

    # ========== BACKLOG MATRIX ==========

    periods, products = demand.shape
    Skip_production = products
    setup_change = products + 1
    setup_change_direct = products + 2
    solution = solution_full[:, :-3]

    # Create the backlog matrix and a matrix for the cumulative demand
    backlog = np.zeros((periods, products), dtype=np.int32)
    cum_demand = np.zeros(products, dtype=np.int32)

    # For each period we add the demand to the cum_demand array which subsequently gets added to the backlog matrix
    # The backlog matrix shows (beginning with the last period) how many products still need to be produced
    for per in range(periods):
        src = periods - per - 1
        for prod in range(products):
            cum_demand[prod] += demand[src, prod]
        for prod in range(products):
            backlog[src, prod] = cum_demand[prod]

    # ========== LAST PERIOD ==========
            
    # Create a matrix for the decoded solution
    decoded = np.zeros((periods + 1, 3), dtype=np.int32)
    

    # Get the index of the last period
    period = periods - 1

    # Get the valid products (those with backlog > 0)
    valid_products = np.full(products, -1.0)


    # If there is demand in the previous period, but not in this one, we can change the setup
    if np.all(backlog[period] == 0):
        for p in range(products):
            valid_products[p] = solution[period, p]
    else:
        for p in range(products):
            if backlog[period, p] > 0:
                valid_products[p] = solution[period, p]

    # Select the product with the highest priority
    second_product = np.argmax(valid_products)
    decoded[period+1, 2] = second_product


    capacity = capacities[period]

    # If the binary indicator for a setup change is 1, directly plan the setup change so that we do not violate capacity restrictions
    if solution_full[period, setup_change] == 1:

        # Valid products this time also include those from the next period
        back = backlog[period] + backlog[period - 1]
        valid_products_2 = np.full(products, -1.0)
        for p in range(products):
            if back[p] > 0:
                valid_products_2[p] = solution[period, p]

        # We set the value of the first product to -1, so that it cannot be selected
        valid_products_2[second_product] = -1
        first_product = np.argmax(valid_products_2)

        # Set the first product and subtract the setup times from the capcity
        decoded[period, 2] = first_product
        capacity -= setup_times[first_product, second_product]

        # Produce the second product either until the demand or capacity is satisfied
        max_quantity = capacity // production_times[second_product]
        quantity = min(max_quantity, backlog[period, second_product])
        decoded[period + 1, 0] = quantity
        for j in range(periods):
            backlog[j, second_product] -= quantity

        capacity -= quantity * production_times[second_product]

        # Produce the first product if there is spare capacity
        if capacity > 0 and solution_full[period, Skip_production] == 0:
            max_quantity = capacity // production_times[first_product]
            quantity = min(max_quantity, backlog[period, first_product])
            decoded[period, 1] = quantity
            for j in range(periods):
                backlog[j, first_product] -= quantity

    # If we do not change the setup, just produce the second product 
    else:
        max_quantity = capacity // production_times[second_product]
        quantity = min(max_quantity, backlog[period, second_product])
        decoded[period + 1, 0] = quantity
        for j in range(periods):
            backlog[j, second_product] -= quantity

        decoded[period, 2] = second_product



    # ========== OTHER PERIODS ==========
            
    for i in range(len(solution) - 1):

        # Start with the second to last period and work to the beginning
        period = periods - 2 - i

        # Get the valid products (those with backlog > 0)
        valid_products = np.full(products, -1.0)
        # If there is no demand in this or the previous period, we do not need to produce or change the setup
        if np.all(backlog[period] + backlog[period-1] == 0):
            decoded[period,2] = decoded[period+1,2]
            continue


        # If there is demand in the previous period, but not in this one, we can change the setup
        elif np.all(backlog[period] == 0):
            for p in range(products):
                valid_products[p] = solution[period, p]
        else:
            for p in range(products):
                if backlog[period, p] > 0:
                    valid_products[p] = solution[period, p]

        # Select the product with the highest priority
        second_product = np.argmax(valid_products)

        # If the product with the highest priority is the same as in the next period, we do not need to change the setup
        if second_product == decoded[period + 1, 2]:
            capacity = capacities[period]

            # If the binary indicator for a setup change is 1, directly plan the setup change so that we do not violate capacity restrictions
            if solution_full[period, setup_change] == 1:

                # Valid products this time also include those from the next period
                back = backlog[period] + backlog[period - 1]
                valid_products_2 = np.full(products, -1.0)
                for p in range(products):
                    if back[p] > 0:
                        valid_products_2[p] = solution[period, p]

                # We set the value of the first product to -1, so that it cannot be selected
                valid_products_2[second_product] = -1
                first_product = np.argmax(valid_products_2)

                # Set the first product and subtract the setup times from the capcity
                decoded[period, 2] = first_product
                capacity -= setup_times[first_product, second_product]

                # Produce the second product either until the demand or capacity is satisfied
                max_quantity = capacity // production_times[second_product]
                quantity = min(max_quantity, backlog[period, second_product])
                decoded[period + 1, 0] = quantity
                for j in range(periods):
                    backlog[j, second_product] -= quantity

                capacity -= quantity * production_times[second_product]

                # Produce the first product if there is spare capacity
                if capacity > 0 and solution_full[period, Skip_production] == 0:
                    max_quantity = capacity // production_times[first_product]
                    quantity = min(max_quantity, backlog[period, first_product])
                    decoded[period, 1] = quantity
                    for j in range(periods):
                        backlog[j, first_product] -= quantity

            # If we do not change the setup, just produce the second product 
            else:
                max_quantity = capacity // production_times[second_product]
                quantity = min(max_quantity, backlog[period, second_product])
                decoded[period + 1, 0] = quantity
                for j in range(periods):
                    backlog[j, second_product] -= quantity

                decoded[period, 2] = second_product

        elif solution_full[period, setup_change_direct] == 1:

            if backlog[period, decoded[period+1, 2]] > 0:
                second_product = decoded[period+1, 2]
                decoded[period, 2] = second_product
                spare_capacity = capacities[period] - setup_times[second_product, decoded[period + 1, 2]]
                max_quantity = spare_capacity // production_times[second_product]
                quantity = min(max_quantity, backlog[period, second_product])
                decoded[period, 1] = quantity
                for j in range(periods):
                    backlog[j, second_product] -= quantity

            elif solution_full[period, Skip_production] == 0:
                decoded[period, 2] = second_product
                spare_capacity = capacities[period] - setup_times[second_product, decoded[period + 1, 2]]
                max_quantity = spare_capacity // production_times[second_product]
                quantity = min(max_quantity, backlog[period, second_product])
                decoded[period, 1] = quantity
                for j in range(periods):
                    backlog[j, second_product] -= quantity

            else:
                decoded[period, 2] = decoded[period + 1, 2]


        # If the product with the highest priority differs from the one in the last period, change the setup directly
        elif solution_full[period, Skip_production] == 0:
            decoded[period, 2] = second_product
            spare_capacity = capacities[period] - setup_times[second_product, decoded[period + 1, 2]]
            max_quantity = spare_capacity // production_times[second_product]
            quantity = min(max_quantity, backlog[period, second_product])
            decoded[period, 1] = quantity
            for j in range(periods):
                backlog[j, second_product] -= quantity

        # If we skip production, just transfer the setup from the last period
        else:
            decoded[period, 2] = decoded[period + 1, 2]

    # If the backlog matrix is > 0, the solution is infeasible and must be punished
    shortage = np.sum(backlog[0]) * penalty_shortage

    # ========== CALCULATE FITNESS ==========
    lots1 = decoded[1:, 0]
    lots2 = decoded[:-1, 1]
    seq = decoded[:, 2]

    # Create a matrix with the production quantites (Periods x Products)
    production = np.zeros((periods, products), dtype=np.int32)
    for t in range(periods):
        production[t, seq[t + 1]] += lots1[t]
        production[t, seq[t]] += lots2[t]

    # Add the production quantities up to get the inventory matrix (Periods x Products)
    inventory = np.zeros((periods, products), dtype=np.int32)
    for m in range(products):
        cum = 0
        for t in range(periods):
            cum += production[t, m] - demand[t, m]
            inventory[t, m] = cum

    # Calaculate holding and shortage costs
    hold_cost = 0
    for t in range(periods):
        for m in range(products):
            inv = inventory[t, m]
            hold_cost += max(inv,0) * holding_costs[m]

    #print(inventory)

    setup_cost = 0
    for t in range(periods):
        setup_cost += setup_costs[seq[t], seq[t + 1]]

    return hold_cost + shortage + setup_cost

def evaluate_individual_new(
    solution,  # shape: (periods+1, 3)
    demand,
    holding_costs,
    setup_costs,
    setup_times,
    production_times,
    capacity_per_period,
    penalty_shortage,
    penalty_overtime
):
    periods, products = demand.shape
    periods_plus1 = periods + 1

    lots1 = solution[1:, 0].astype(np.int32)
    lots2 = solution[:-1, 1].astype(np.int32)
    seq = solution[:, 2].astype(np.int32)

    # --- Production matrix ---
    production = np.zeros((periods, products), dtype=np.int32)
    for t in range(periods):
        production[t, seq[t + 1]] += lots1[t]
        production[t, seq[t]] += lots2[t]

    print("\n=== Production Matrix ===")
    print(production)

    # --- Inventory matrix ---
    inventory = np.zeros((periods, products), dtype=np.float64)
    for m in range(products):
        cum = 0.0
        for t in range(periods):
            cum += production[t, m] - demand[t, m]
            inventory[t, m] = cum

    print("\n=== Inventory Matrix ===")
    print(inventory)

    # --- Holding and shortage costs ---
    hold_costs_per_product = np.sum(inventory * holding_costs, axis=0)
    hold_cost_total = np.sum(hold_costs_per_product)

    shortage_violations = inventory < 0
    shortage_units = -np.sum(inventory[shortage_violations])
    shortage_penalty = shortage_units * penalty_shortage

    print("\n--- Holding Costs per Product ---")
    for i in range(products):
        print(f"Product {i}: {hold_costs_per_product[i]:.2f}")

    print("\nTotal Holding Cost:", hold_cost_total)
    print("Total Shortage Penalty:", shortage_penalty)
    print("Total Shortage Violations (units):", shortage_units)

    # --- Setup and production time ---
    setup_time = np.zeros(periods, dtype=np.int32)
    for t in range(periods):
        setup_time[t] = setup_times[seq[t], seq[t + 1]]

    prod_time = np.sum(production * production_times, axis=1)
    total_time = prod_time + setup_time
    overtime_violations = np.maximum(total_time - capacity_per_period, 0)
    overtime_penalty = np.sum(overtime_violations) * penalty_overtime

    print("\nSetup Times per Period:", setup_time)
    print("Production Times per Period:", prod_time)
    print("Total Time per Period:", total_time)
    print("Overtime Violations (units):", np.sum(overtime_violations))
    print("Overtime Penalty:", overtime_penalty)

    # --- Setup cost ---
    setup_c = 0.0
    for t in range(periods):
        setup_c += setup_costs[seq[t], seq[t + 1]]

    print("\nTotal Setup Cost:", setup_c)

    total_cost = hold_cost_total + shortage_penalty + overtime_penalty + setup_c
    print("\n=== Final Total Cost ===")
    print(total_cost)

@njit(cache=True)
def create_starting_population(demand, num_individuals, num_children,
                                prob_zero_prod, prob_setup_change,num_subpopulations:int,prob_direct_setup_change):
    periods, products = demand.shape
    total_individuals = num_individuals + num_children
    population = np.zeros((num_subpopulations, total_individuals, periods, products + 3), dtype=np.float32)

    for pop in range(num_subpopulations):
        for ind in range(num_individuals):
            for row in range(periods):
                for col in range(products + 3):
                    if col < products:
                        population[pop, ind, row, col] = np.random.rand()
                    elif col == products:  # second-to-last column (zero-prod periods)
                        population[pop, ind, row, col] = 1.0 if np.random.rand() >= prob_zero_prod else 0.0
                    elif col == products+1:  # last column (setup change allowed)
                        population[pop, ind, row, col] = 1.0 if np.random.rand() >= prob_setup_change else 0.0

                    else:
                        population[pop, ind, row, col] = 1.0 if np.random.rand() >= prob_direct_setup_change else 0.0

    return population


# ========== CROSSOVER FUNCTIONS ==========

@njit
def select_parents_subpop(fitness: np.ndarray, num_children: int) -> tuple:
    inverse_fitness = 1.0 / fitness
    num_inds = fitness.shape[0]
    probabilities = inverse_fitness / np.sum(inverse_fitness)

    # Cumulative probabilities
    cumulative = np.empty(num_inds, dtype=np.float64)
    cumulative[0] = probabilities[0]
    for i in range(1, num_inds):
        cumulative[i] = cumulative[i - 1] + probabilities[i]

    # Draw random numbers
    rand_vals = np.random.rand(num_children * 2)
    indices = np.empty(num_children * 2, dtype=np.int32)

    for i in range(num_children * 2):
        r = rand_vals[i]
        for j in range(num_inds):
            if r <= cumulative[j]:
                indices[i] = j
                break

    return indices[:num_children], indices[num_children:]


@njit
def two_point_crossover(population, parents1_idx, parents2_idx, num_children):
    total_individuals, periods, genes = population.shape
    start_idx = total_individuals - num_children  # where to begin writing children

    for i in range(num_children):
        p1 = population[parents1_idx[i]]
        p2 = population[parents2_idx[i]]

        # Random crossover points (ensure point1 < point2)
        point1 = np.random.randint(0, periods - 1)
        point2 = np.random.randint(point1 + 1, periods)

        child1 = population[start_idx + i]
        child2 = population[start_idx + i]  # same memory, written complementarily

        for t in range(periods):
            if point1 <= t < point2:
                # Inside crossover window
                child1[t] = p2[t]
                child2[t] = p1[t]
            else:
                # Outside crossover window
                child1[t] = p1[t]
                child2[t] = p2[t]

# ========== MUTATION FUNCTIONS ==========

@njit
def mutate_priorities_old(individual, prob: float, strength: float = 0.1):
    periods, genes = individual.shape
    for i in range(periods):
        for j in range(genes): 
            if np.random.rand() < prob:
                delta = (np.random.rand() * 2 - 1) * strength  # ∈ [-strength, strength]
                individual[i, j] += delta
                # Clip to [0, 1]
                if individual[i, j] > 1.0:
                    individual[i, j] = 1.0
                elif individual[i, j] < 0.0:
                    individual[i, j] = 0.0

@njit
def mutate_priorities(individual, prob: float, strength: float = 0.1):
    periods, genes = individual.shape
    priority_genes = genes - 3  # Last 2 columns are binary

    for i in range(periods):
        for j in range(priority_genes):
            if np.random.rand() < prob:
                delta = (np.random.rand() * 2 - 1) * strength  # ∈ [-strength, strength]
                individual[i, j] += delta
                # Clip to [0, 1]
                #if individual[i, j] > 1.0:
                    #individual[i, j] = 1.0
                if individual[i, j] < 0.0:
                    individual[i, j] = 0.0

@njit
def mutate_skip_flags(individual, prob_skip: float):
    periods, genes = individual.shape
    for i in range(periods):
        if np.random.rand() < prob_skip:
            individual[i, genes-3] = 1.0 - individual[i, genes-3]  # flip 0/1

@njit
def mutate_setup_flags(individual, prob_setup: float):
    periods, genes = individual.shape
    for i in range(periods):
        if np.random.rand() < prob_setup:
            individual[i, genes-2] = 1.0 - individual[i, genes-2]  # flip 0/1

@njit
def mutate_setup_change_flags(individual, prob_setup: float):
    periods, genes = individual.shape
    for i in range(periods):
        if np.random.rand() < prob_setup:
            individual[i, genes-1] = 1.0 - individual[i, genes-1]  # flip 0/1

# ========== NATURAL SELECTION FUNCTIONS ==========

@njit
def boltzmann_temperature(progress: float, T_max: float = 5.0, T_min: float = 0.1, steepness: float = 8.0) -> float:
    sigmoid = 1.0 / (1.0 + np.exp(-steepness * (progress - 0.6)))
    return T_max - (T_max - T_min) * sigmoid

@njit
def natural_selection_boltzmann(
    solutions: np.ndarray,
    fitness: np.ndarray,
    end_size: int,
    elite_ratio: float,
    temperature: float
):
    N = fitness.shape[0]
    num_elite = int(end_size * elite_ratio)

    # --- Step 1: Elitism ---
    sorted_indices = np.argsort(fitness)
    selected_indices = np.empty(end_size, dtype=np.int32)

    for i in range(num_elite):
        selected_indices[i] = sorted_indices[i]

    # --- Step 2: Boltzmann Selection ---
    min_fit = np.min(fitness)
    max_fit = np.max(fitness)

    normalized = np.empty(N, dtype=np.float64)
    if max_fit - min_fit == 0:
        normalized[:] = 1.0
    else:
        for i in range(N):
            normalized[i] = 1.0 - (fitness[i] - min_fit) / (max_fit - min_fit)

    scores = np.exp(normalized / temperature)
    total_score = np.sum(scores)
    probabilities = scores / total_score

    # Sample additional individuals
    for i in range(num_elite, end_size):
        r = np.random.rand()
        cumulative = 0.0
        for j in range(N):
            cumulative += probabilities[j]
            if r < cumulative:
                selected_indices[i] = j
                break

    # --- Step 3: Copy selected individuals and fitness values ---
    temp_solutions = solutions.copy()
    temp_fitness = fitness.copy()

    for i in range(end_size):
        solutions[i, :, :] = temp_solutions[selected_indices[i], :, :]
        fitness[i] = temp_fitness[selected_indices[i]]

@njit
def natural_selection_boltzmann_old(
    solutions: np.ndarray,
    fitness: np.ndarray,
    end_size: int,
    elite_ratio: float,
    temperature: float
):
    N = fitness.shape[0]
    num_elite = int(end_size * elite_ratio)

    # --- Step 1: Elitism ---
    sorted_indices = np.argsort(fitness)
    for i in range(num_elite):
        elite_idx = sorted_indices[i]
        # Copy elite individuals to the front
        solutions[i, :, :] = solutions[elite_idx, :, :]
        fitness[i] = fitness[elite_idx]

    # --- Step 2: Boltzmann Selection for rest ---
    min_fit = np.min(fitness)
    max_fit = np.max(fitness)

    normalized = np.empty(N, dtype=np.float64)
    if max_fit - min_fit == 0:
        normalized[:] = 1.0
    else:
        for i in range(N):
            normalized[i] = 1.0 - (fitness[i] - min_fit) / (max_fit - min_fit)

    # Boltzmann probabilistic scores
    scores = np.exp(normalized / temperature)
    total_score = np.sum(scores)
    probabilities = scores / total_score

    # --- Random weighted sampling (with replacement) ---
    for i in range(num_elite, end_size):
        r = np.random.rand()
        cumulative = 0.0
        for j in range(N):
            cumulative += probabilities[j]
            if r < cumulative:
                solutions[i, :, :] = solutions[j, :, :]
                fitness[i] = fitness[j]
                break

@njit(cache=True)
def reset_population_inplace(
    demand: np.ndarray,
    solutions: np.ndarray,
    fitness: np.ndarray,
    new_size: int,
    preserve_ratio: float,
    prob_zero_prod: float,
    prob_setup_change: float
):
    periods, products = demand.shape
    num_preserve = max(1, int(new_size * preserve_ratio))

    # --- Step 1: Sort & preserve top elites in-place ---
    elite_indices = np.argsort(fitness[:new_size])[:num_preserve]
    for i in range(num_preserve):
        idx = elite_indices[i]
        if idx != i:
            tmp = np.copy(solutions[idx])
            solutions[i] = tmp
            fitness[i] = fitness[idx]

    # --- Step 2: Overwrite the rest with random individuals ---
    for ind in range(num_preserve, new_size):
        for row in range(periods):
            for col in range(products + 3):
                if col < products:
                    solutions[ind, row, col] = np.random.rand()
                elif col == products:  # zero-prod column
                    solutions[ind, row, col] = 1.0 if np.random.rand() >= prob_zero_prod else 0.0
                else:  # setup change column
                    solutions[ind, row, col] = 1.0 if np.random.rand() >= prob_setup_change else 0.0


# ========== ITERATION FUNCTIONS ========== 

@njit(parallel=True)
def run_parallel_iteration(
    demand: np.ndarray,
    capacities: np.ndarray,
    production_times: np.ndarray,
    setup_times: np.ndarray,
    holding_costs: np.ndarray,
    setup_costs: np.ndarray,
    penalty: int,
    population: np.ndarray,           # shape: (num_subpops, individuals, periods, genes)
    fitness: np.ndarray,              # shape: (num_subpops, individuals)
    stagnation_count: np.ndarray,     # shape: (num_subpops,)
    stagnation_limit: int,
    num_parents: int,
    num_children: int,
    prob_mutate_priority: float,
    strength_mutate_priority: float,
    prob_mutate_setup: float,
    prob_mutate_skip: float,
    prob_mutate_direct_change: float,
    progress: float,
    num_subpopulations: int
):
    for pop in prange(num_subpopulations):
        pop_array = population[pop]
        fit_array = fitness[pop]
        stagnation = stagnation_count[pop]

        # --- Step 1: Selection & Crossover
        idx1, idx2 = select_parents_subpop(fit_array[:num_parents], num_children)
        two_point_crossover(pop_array, idx1, idx2, num_children)

        # --- Step 2: Mutation & Evaluation (for children)
        for i in range(num_parents, num_parents + num_children):
            mutate_priorities(pop_array[i], prob_mutate_priority, strength_mutate_priority)
            mutate_setup_flags(pop_array[i], prob_mutate_setup)
            mutate_skip_flags(pop_array[i], prob_mutate_skip)
            mutate_setup_change_flags(pop_array[i], prob_mutate_direct_change)

            fit_array[i] = decode_num(
                demand, capacities, production_times, setup_times,
                pop_array[i], holding_costs, setup_costs, penalty
            )

        # --- Step 3: Natural Selection
        T = boltzmann_temperature(progress, T_max=50.0, T_min=10, steepness=8.0)
        natural_selection_boltzmann(
            pop_array, fit_array, end_size=num_parents, elite_ratio=0.1, temperature=T
        )

        # --- Step 4: Stagnation Reset
        if stagnation > stagnation_limit:
            reset_population_inplace(
                demand, pop_array, fit_array,
                new_size=num_parents, preserve_ratio=0.2,
                prob_zero_prod=1, prob_setup_change=0.5
            )
            for ind in range(num_parents):
                fit_array[ind] = decode_num(
                    demand, capacities, production_times, setup_times,
                    pop_array[ind], holding_costs, setup_costs, penalty
                )

# ========== NETWORK FUNCTIONS ==========
            
def create_er_network(solutions: np.ndarray, chance=0.2):
    subpops = solutions.shape[0]
    network = np.zeros((subpops, subpops), dtype=np.int32)

    for i in range(subpops):
        for j in range(i + 1, subpops):  
            if np.random.rand() < chance:
                network[i, j] = 1
                network[j, i] = 1  

    for i in range(subpops):
        if not np.any(network[i]):  # no neighbors
            j = np.random.choice([x for x in range(subpops) if x != i])
            network[i, j] = 1
            network[j, i] = 1

    return network

def create_neighbour_network(num_populations):

    network = np.zeros((num_populations,num_populations),dtype = np.int8)

    network[0,num_populations-1] = 1
    network[num_populations-1,0] = 1

    for row in range(10):
        for col in range(10):
            if np.abs(row-col) == 1:
                network[row,col] = 1

    return network

@njit
def update_network(solutions, fitness, network) -> np.ndarray:

    subpops, inds, periods, genes = solutions.shape

    # Step 1: Find elite individual and fitness in each population
    elite_fit = np.zeros(subpops, dtype=np.int32)
    elite_ind = np.zeros((subpops, periods, genes), dtype=solutions.dtype)

    for i in range(subpops):
        elite_idx = np.argmin(fitness[i])
        elite_fit[i] = fitness[i, elite_idx]
        elite_ind[i] = solutions[i, elite_idx]

    # Step 2: Select random subpopulation
    selected_pop = np.random.randint(subpops)

    # Step 3: Get neighbors from the network
    neighbors = np.where(network[selected_pop] == 1)[0]
    if len(neighbors) == 0:
        return  # nothing to update

    # Step 4: Find the best individual among neighbors
    best_neighbor_idx = neighbors[np.argmin(elite_fit[neighbors])]
    best_individual = elite_ind[best_neighbor_idx]
    best_fitness = elite_fit[best_neighbor_idx]

    # Step 5: Replace first individual in all neighbors
    for i in neighbors:
        solutions[i, 0] = best_individual
        fitness[i, 0] = best_fitness

def warmup_numba(problem, num_individuals, num_children, penalty, num_subpopulations):
    print("Compiling Functions ...")

    periods, products = problem.demand.shape
    dummy_pop = create_starting_population(problem.demand, num_individuals, num_children,1,0.5,num_subpopulations,0.9)
    dummy_fit = np.zeros((num_subpopulations, num_individuals + num_children), dtype=np.int32)
    dummy_stag = np.zeros(num_subpopulations, dtype=np.int32)

    # Dummy progress
    progress = 0.5

    # Warm up parallel iteration
    run_parallel_iteration(problem.demand, problem.capacities, problem.production_times, problem.setup_times,
                           problem.holding_costs, problem.setup_costs, penalty, dummy_pop, dummy_fit,
                           dummy_stag, stagnation_limit=5, num_parents=num_individuals,
                           num_children=num_children, prob_mutate_priority=0.03, strength_mutate_priority=1.0,
                           prob_mutate_setup=0.01, prob_mutate_skip=0.01,prob_mutate_direct_change=0.03,
                           progress=progress, num_subpopulations=num_subpopulations)

    # Warm up network update
    net = create_er_network(dummy_pop, 0.2)
    update_network(dummy_pop, dummy_fit, net)
    
# ========== MAIN FUNCTION ==========

def main(problem:ProblemInstance,num_individuals:int,
         penalty:int,num_children:int,stagnation_limit:int,num_subpopulations:int):
    
    #========== COMPILE FUNCTIONS ==========
    warmup_numba(problem,num_individuals,num_children,penalty,num_subpopulations)   


    #========== MAIN ALGORITHM ==========

    # Create the starting population and evaluate it
    solutions = create_starting_population( problem.demand, num_individuals, num_children,1,0.5,num_subpopulations,0.9)

    # Create an array to store the fitness values
    fitness = np.zeros((num_subpopulations, num_individuals+num_children),dtype=np.int32)

    for pop in range(num_subpopulations):
        for ind in range(num_individuals+num_children):

            fitness[pop,ind] = decode_num(
                    problem.demand, problem.capacities, problem.production_times, problem.setup_times,
                    solutions[pop,ind], problem.holding_costs, problem.setup_costs, penalty
                )

    # Store the Population
    algorithm = Population(
        population = solutions,
        fitness = fitness
    )

    # Initialize metrics
    iteration = 0
    stagnation_count = np.zeros(num_subpopulations)
    time_limit = 90
    elapsed_time = 0
    start_time = time.time()

    # Create the ER Network
    network = create_er_network(algorithm.population,0.1)
    #+network = create_neighbour_network(num_subpopulations)

    # Create a progress bar
    progress = tqdm(total=time_limit, desc="Optimizing", dynamic_ncols=True)


    while elapsed_time <= time_limit:

        # Update the progess
        prog = elapsed_time/time_limit


        # Run the main algorithm
        run_parallel_iteration(problem.demand,problem.capacities,problem.production_times,problem.setup_times,
                      problem.holding_costs,problem.setup_costs,penalty,algorithm.population,algorithm.fitness
                      ,stagnation_count,stagnation_limit,num_individuals,num_children,num_subpopulations=num_subpopulations,strength_mutate_priority=0.8,progress=prog,
                      prob_mutate_priority=0.02,
                      prob_mutate_setup=0.03,
                      prob_mutate_skip=0.01,
                      prob_mutate_direct_change=0.02) #0.03,0.02,0.01

        # Update Iterations and Time
        elapsed_time = round(time.time() - start_time,0)
        progress.update(elapsed_time - progress.n)
        iteration += 1

        if iteration % 50 == 0:
            update_network(algorithm.population,algorithm.fitness,network)

        current_best = np.min(algorithm.fitness)

        # Update the Progress bar
        progress.set_description(f"Iteration: {iteration+1}, Best: {current_best:.0f}")

    final_result_idx = np.argmin(algorithm.fitness)
    final_result_idx = np.unravel_index(final_result_idx, algorithm.fitness.shape)
    final_result = algorithm.population[final_result_idx]

    result = decode_verbose(problem.demand,problem.capacities,problem.production_times,problem.setup_times,
               final_result,problem.holding_costs,problem.setup_costs,penalty)
    
    idx = np.unravel_index(np.argmin(algorithm.fitness), algorithm.fitness.shape)
    best = algorithm.population[idx]
    


    return best, algorithm

# ========== TABU FUNCTIONS ==========
def hash_solution(solution: np.ndarray) -> int:
    # Ensure consistent and efficient hashing
    return hash(solution.tobytes())

class TabuList:
    def __init__(self, max_size: int):
        self.max_size = max_size
        self.queue = deque()
        self.hashes = set()

    def add(self, solution: np.ndarray):
        h = hash_solution(solution)
        if h in self.hashes:
            return  

        if len(self.queue) >= self.max_size:
            old = self.queue.popleft()
            self.hashes.discard(old) 

        self.queue.append(h)
        self.hashes.add(h)

        # Debug check (optional, remove in production for speed)
        assert len(self.queue) == len(self.hashes), "Tabu queue and hash set out of sync!"

    def is_tabu(self, solution: np.ndarray) -> bool:
        h = hash_solution(solution)
        return h in self.hashes

@njit
def decode_prod_plan(demand, capacities, production_times, setup_times,
           solution_full, holding_costs, setup_costs, penalty_shortage=1):

    # ========== BACKLOG MATRIX ==========

    periods, products = demand.shape
    Skip_production = products
    setup_change = products + 1
    setup_change_direct = products + 2
    solution = solution_full[:, :-3]

    # Create the backlog matrix and a matrix for the cumulative demand
    backlog = np.zeros((periods, products), dtype=np.int32)
    cum_demand = np.zeros(products, dtype=np.int32)

    # For each period we add the demand to the cum_demand array which subsequently gets added to the backlog matrix
    # The backlog matrix shows (beginning with the last period) how many products still need to be produced
    for per in range(periods):
        src = periods - per - 1
        for prod in range(products):
            cum_demand[prod] += demand[src, prod]
        for prod in range(products):
            backlog[src, prod] = cum_demand[prod]

    # ========== LAST PERIOD ==========
            
    # Create a matrix for the decoded solution
    decoded = np.zeros((periods + 1, 3), dtype=np.int32)
    

    # Get the index of the last period
    period = periods - 1

    # Get the valid products (those with backlog > 0)
    valid_products = np.full(products, -1.0)


    # If there is demand in the previous period, but not in this one, we can change the setup
    if np.all(backlog[period] == 0):
        for p in range(products):
            valid_products[p] = solution[period, p]
    else:
        for p in range(products):
            if backlog[period, p] > 0:
                valid_products[p] = solution[period, p]

    # Select the product with the highest priority
    second_product = np.argmax(valid_products)
    decoded[period+1, 2] = second_product


    capacity = capacities[period]

    # If the binary indicator for a setup change is 1, directly plan the setup change so that we do not violate capacity restrictions
    if solution_full[period, setup_change] == 1:

        # Valid products this time also include those from the next period
        back = backlog[period] + backlog[period - 1]
        valid_products_2 = np.full(products, -1.0)
        for p in range(products):
            if back[p] > 0:
                valid_products_2[p] = solution[period, p]

        # We set the value of the first product to -1, so that it cannot be selected
        valid_products_2[second_product] = -1
        first_product = np.argmax(valid_products_2)

        # Set the first product and subtract the setup times from the capcity
        decoded[period, 2] = first_product
        capacity -= setup_times[first_product, second_product]

        # Produce the second product either until the demand or capacity is satisfied
        max_quantity = capacity // production_times[second_product]
        quantity = min(max_quantity, backlog[period, second_product])
        decoded[period + 1, 0] = quantity
        for j in range(periods):
            backlog[j, second_product] -= quantity

        capacity -= quantity * production_times[second_product]

        # Produce the first product if there is spare capacity
        if capacity > 0 and solution_full[period, Skip_production] == 0:
            max_quantity = capacity // production_times[first_product]
            quantity = min(max_quantity, backlog[period, first_product])
            decoded[period, 1] = quantity
            for j in range(periods):
                backlog[j, first_product] -= quantity

    # If we do not change the setup, just produce the second product 
    else:
        max_quantity = capacity // production_times[second_product]
        quantity = min(max_quantity, backlog[period, second_product])
        decoded[period + 1, 0] = quantity
        for j in range(periods):
            backlog[j, second_product] -= quantity

        decoded[period, 2] = second_product



    # ========== OTHER PERIODS ==========
            
    for i in range(len(solution) - 1):

        # Start with the second to last period and work to the beginning
        period = periods - 2 - i

        # Get the valid products (those with backlog > 0)
        valid_products = np.full(products, -1.0)
        # If there is no demand in this or the previous period, we do not need to produce or change the setup
        if np.all(backlog[period] + backlog[period-1] == 0):
            decoded[period,2] = decoded[period+1,2]
            continue


        # If there is demand in the previous period, but not in this one, we can change the setup
        elif np.all(backlog[period] == 0):
            for p in range(products):
                valid_products[p] = solution[period, p]
        else:
            for p in range(products):
                if backlog[period, p] > 0:
                    valid_products[p] = solution[period, p]

        # Select the product with the highest priority
        second_product = np.argmax(valid_products)

        # If the product with the highest priority is the same as in the next period, we do not need to change the setup
        if second_product == decoded[period + 1, 2]:
            capacity = capacities[period]

            # If the binary indicator for a setup change is 1, directly plan the setup change so that we do not violate capacity restrictions
            if solution_full[period, setup_change] == 1:

                # Valid products this time also include those from the next period
                back = backlog[period] + backlog[period - 1]
                valid_products_2 = np.full(products, -1.0)
                for p in range(products):
                    if back[p] > 0:
                        valid_products_2[p] = solution[period, p]

                # We set the value of the first product to -1, so that it cannot be selected
                valid_products_2[second_product] = -1
                first_product = np.argmax(valid_products_2)

                # Set the first product and subtract the setup times from the capcity
                decoded[period, 2] = first_product
                capacity -= setup_times[first_product, second_product]

                # Produce the second product either until the demand or capacity is satisfied
                max_quantity = capacity // production_times[second_product]
                quantity = min(max_quantity, backlog[period, second_product])
                decoded[period + 1, 0] = quantity
                for j in range(periods):
                    backlog[j, second_product] -= quantity

                capacity -= quantity * production_times[second_product]

                # Produce the first product if there is spare capacity
                if capacity > 0 and solution_full[period, Skip_production] == 0:
                    max_quantity = capacity // production_times[first_product]
                    quantity = min(max_quantity, backlog[period, first_product])
                    decoded[period, 1] = quantity
                    for j in range(periods):
                        backlog[j, first_product] -= quantity

            # If we do not change the setup, just produce the second product 
            else:
                max_quantity = capacity // production_times[second_product]
                quantity = min(max_quantity, backlog[period, second_product])
                decoded[period + 1, 0] = quantity
                for j in range(periods):
                    backlog[j, second_product] -= quantity

                decoded[period, 2] = second_product

        elif solution_full[period, setup_change_direct] == 1:

            if backlog[period, decoded[period+1, 2]] > 0:
                second_product = decoded[period+1, 2]
                decoded[period, 2] = second_product
                spare_capacity = capacities[period] - setup_times[second_product, decoded[period + 1, 2]]
                max_quantity = spare_capacity // production_times[second_product]
                quantity = min(max_quantity, backlog[period, second_product])
                decoded[period, 1] = quantity
                for j in range(periods):
                    backlog[j, second_product] -= quantity

            elif solution_full[period, Skip_production] == 0:
                decoded[period, 2] = second_product
                spare_capacity = capacities[period] - setup_times[second_product, decoded[period + 1, 2]]
                max_quantity = spare_capacity // production_times[second_product]
                quantity = min(max_quantity, backlog[period, second_product])
                decoded[period, 1] = quantity
                for j in range(periods):
                    backlog[j, second_product] -= quantity

            else:
                decoded[period, 2] = decoded[period + 1, 2]


        # If the product with the highest priority differs from the one in the last period, change the setup directly
        elif solution_full[period, Skip_production] == 0:
            decoded[period, 2] = second_product
            spare_capacity = capacities[period] - setup_times[second_product, decoded[period + 1, 2]]
            max_quantity = spare_capacity // production_times[second_product]
            quantity = min(max_quantity, backlog[period, second_product])
            decoded[period, 1] = quantity
            for j in range(periods):
                backlog[j, second_product] -= quantity

        # If we skip production, just transfer the setup from the last period
        else:
            decoded[period, 2] = decoded[period + 1, 2]

    return decoded

def run_tabu_search(problem:ProblemInstance,
                    start_solution:np.ndarray,
                    start_solution_prod:np.ndarray,
                    start_obj_value:int,
                    len_tabu_list:int,
                    time_limit:int,
                    penalty:int,
                    periods_to_modify:int):

    # Create a variable that always stores the current best solution/objective value
    best_solution = start_solution
    best_objective_value = start_obj_value

    # Store the current solution
    current_solution = start_solution

    # Create the Tabu List
    tabu_list = TabuList(len_tabu_list)
    tabu_list.add(start_solution_prod)

    # Get the number of periods and products
    periods = problem.num_periods
    products = problem.num_products

    # Set a timer
    start_time = time.time()
    elapsed_time = 0
    iter = 0

    # Create a progress bar
    progress = tqdm(total=time_limit, desc="Optimizing", dynamic_ncols=True)

    # Calculate the size of the neighbourhood
    neighbourhood_size = periods_to_modify * (products * 8)

    while elapsed_time < time_limit:

        # Update the iteration count
        iter +=1

        # Run one iteration of the algorithm
        best_neighbour, best_neighbour_fit, valid = run_tabu_iteration(problem,current_solution,neighbourhood_size,penalty,tabu_list,periods_to_modify)

        # Add the best neighbour to the tabu list
        best_neighbour_prod, best_neighbour_decode,_ = decode_full(problem.demand,
                                                 problem.capacities,
                                                 problem.production_times,
                                                 problem.setup_times,
                                                 current_solution,
                                                 problem.holding_costs,
                                                 problem.setup_costs,
                                                 penalty)       
        if valid:
            current_solution = best_neighbour
            tabu_list.add(best_neighbour_prod)

        # Update the global best, if the best neighbour is better
        if best_neighbour_fit < best_objective_value and valid:
            best_objective_value = best_neighbour_fit
            best_solution = current_solution

        # Update the time
        elapsed_time = time.time() - start_time


        # Update the progress bar
        progress.update(1)
        if not valid:
            progress.set_description(f"Iteration: {iter+1}, No valid move, Best: {best_objective_value:.0f}")
        else:
            progress.set_description(f"Iteration: {iter+1}, Best: {best_objective_value:.0f}, Current: {best_neighbour_fit:.0f}")

    
    decode_verbose(problem.demand,problem.capacities,problem.production_times,problem.setup_times,
                    best_solution,problem.holding_costs,problem.setup_costs,penalty)     
    
    return tabu_list


def run_tabu_iteration(problem: ProblemInstance,
                       solution: np.ndarray,
                       neighbourhood_size: int,
                       penalty: float,
                       tabu_list: TabuList,
                       num_periods_to_modify: int = 3):

    scratch = np.empty_like(solution)
    num_periods = problem.num_periods
    num_products = problem.num_products

    neighbourhood = np.zeros((neighbourhood_size, num_periods, num_products + 3), dtype=np.float64)
    neighbourhood_fit = np.zeros(neighbourhood_size, dtype=np.float64)
    valid_count = 0

    periods_to_mutate = np.random.choice(num_periods, size=num_periods_to_modify, replace=False)

    for period in periods_to_mutate:
        row = solution[period]

        # Priority swaps
        for prod in range(num_products):
            np.copyto(scratch, solution)
            prio_row = scratch[period, :-3]
            max_prio = prio_row.argmax()
            if max_prio != prod:
                prio_row[prod], prio_row[max_prio] = prio_row[max_prio], prio_row[prod]

                scratch_prod, scratch_decode, fitness = decode_full(problem.demand, problem.capacities,
                    problem.production_times, problem.setup_times, scratch,
                    problem.holding_costs, problem.setup_costs, penalty)

                if not tabu_list.is_tabu(scratch_prod):
                    neighbourhood[valid_count] = scratch
                    neighbourhood_fit[valid_count] = fitness
                    valid_count += 1
                    if valid_count >= neighbourhood_size:
                        break

                if valid_count >= neighbourhood_size:
                    break

        # If no priority was changed, still explore full bitspace
        for bits in product([0.0, 1.0], repeat=3):
            np.copyto(scratch, solution)
            scratch[period, -3:] = bits
            scratch_prod, scratch_decode, fitness = decode_full(problem.demand, problem.capacities,
                problem.production_times, problem.setup_times, scratch,
                problem.holding_costs, problem.setup_costs, penalty)

            if not tabu_list.is_tabu(scratch_prod):
                neighbourhood[valid_count] = scratch
                neighbourhood_fit[valid_count] = fitness
                valid_count += 1
                if valid_count >= neighbourhood_size:
                    break

        if valid_count >= neighbourhood_size:
            break

    if valid_count == 0:
        _, _, current_fit = decode_full(problem.demand, problem.capacities,
                                        problem.production_times, problem.setup_times,
                                        solution, problem.holding_costs, problem.setup_costs, penalty)
        return solution.copy(), current_fit, False

    idx = np.argmin(neighbourhood_fit[:valid_count])
    return neighbourhood[idx], neighbourhood_fit[idx], True


@njit
def decode_full(demand, capacities, production_times, setup_times,
           solution_full, holding_costs, setup_costs, penalty_shortage=1):

    # ========== BACKLOG MATRIX ==========

    periods, products = demand.shape
    Skip_production = products
    setup_change = products + 1
    setup_change_direct = products + 2
    solution = solution_full[:, :-3]

    # Create the backlog matrix and a matrix for the cumulative demand
    backlog = np.zeros((periods, products), dtype=np.int32)
    cum_demand = np.zeros(products, dtype=np.int32)

    # For each period we add the demand to the cum_demand array which subsequently gets added to the backlog matrix
    # The backlog matrix shows (beginning with the last period) how many products still need to be produced
    for per in range(periods):
        src = periods - per - 1
        for prod in range(products):
            cum_demand[prod] += demand[src, prod]
        for prod in range(products):
            backlog[src, prod] = cum_demand[prod]

    # ========== LAST PERIOD ==========
            
    # Create a matrix for the decoded solution
    decoded = np.zeros((periods + 1, 3), dtype=np.int32)
    

    # Get the index of the last period
    period = periods - 1

    # Get the valid products (those with backlog > 0)
    valid_products = np.full(products, -1.0)


    # If there is demand in the previous period, but not in this one, we can change the setup
    if np.all(backlog[period] == 0):
        for p in range(products):
            valid_products[p] = solution[period, p]
    else:
        for p in range(products):
            if backlog[period, p] > 0:
                valid_products[p] = solution[period, p]

    # Select the product with the highest priority
    second_product = np.argmax(valid_products)
    decoded[period+1, 2] = second_product


    capacity = capacities[period]

    # If the binary indicator for a setup change is 1, directly plan the setup change so that we do not violate capacity restrictions
    if solution_full[period, setup_change] == 1:

        # Valid products this time also include those from the next period
        back = backlog[period] + backlog[period - 1]
        valid_products_2 = np.full(products, -1.0)
        for p in range(products):
            if back[p] > 0:
                valid_products_2[p] = solution[period, p]

        # We set the value of the first product to -1, so that it cannot be selected
        valid_products_2[second_product] = -1
        first_product = np.argmax(valid_products_2)

        # Set the first product and subtract the setup times from the capcity
        decoded[period, 2] = first_product
        capacity -= setup_times[first_product, second_product]

        # Produce the second product either until the demand or capacity is satisfied
        max_quantity = capacity // production_times[second_product]
        quantity = min(max_quantity, backlog[period, second_product])
        decoded[period + 1, 0] = quantity
        for j in range(periods):
            backlog[j, second_product] -= quantity

        capacity -= quantity * production_times[second_product]

        # Produce the first product if there is spare capacity
        if capacity > 0 and solution_full[period, Skip_production] == 0:
            max_quantity = capacity // production_times[first_product]
            quantity = min(max_quantity, backlog[period, first_product])
            decoded[period, 1] = quantity
            for j in range(periods):
                backlog[j, first_product] -= quantity

    # If we do not change the setup, just produce the second product 
    else:
        max_quantity = capacity // production_times[second_product]
        quantity = min(max_quantity, backlog[period, second_product])
        decoded[period + 1, 0] = quantity
        for j in range(periods):
            backlog[j, second_product] -= quantity

        decoded[period, 2] = second_product



    # ========== OTHER PERIODS ==========
            
    for i in range(len(solution) - 1):

        # Start with the second to last period and work to the beginning
        period = periods - 2 - i

        # Get the valid products (those with backlog > 0)
        valid_products = np.full(products, -1.0)
        # If there is no demand in this or the previous period, we do not need to produce or change the setup
        if np.all(backlog[period] + backlog[period-1] == 0):
            decoded[period,2] = decoded[period+1,2]
            continue


        # If there is demand in the previous period, but not in this one, we can change the setup
        elif np.all(backlog[period] == 0):
            for p in range(products):
                valid_products[p] = solution[period, p]
        else:
            for p in range(products):
                if backlog[period, p] > 0:
                    valid_products[p] = solution[period, p]

        # Select the product with the highest priority
        second_product = np.argmax(valid_products)

        # If the product with the highest priority is the same as in the next period, we do not need to change the setup
        if second_product == decoded[period + 1, 2]:
            capacity = capacities[period]

            # If the binary indicator for a setup change is 1, directly plan the setup change so that we do not violate capacity restrictions
            if solution_full[period, setup_change] == 1:

                # Valid products this time also include those from the next period
                back = backlog[period] + backlog[period - 1]
                valid_products_2 = np.full(products, -1.0)
                for p in range(products):
                    if back[p] > 0:
                        valid_products_2[p] = solution[period, p]

                # We set the value of the first product to -1, so that it cannot be selected
                valid_products_2[second_product] = -1
                first_product = np.argmax(valid_products_2)

                # Set the first product and subtract the setup times from the capcity
                decoded[period, 2] = first_product
                capacity -= setup_times[first_product, second_product]

                # Produce the second product either until the demand or capacity is satisfied
                max_quantity = capacity // production_times[second_product]
                quantity = min(max_quantity, backlog[period, second_product])
                decoded[period + 1, 0] = quantity
                for j in range(periods):
                    backlog[j, second_product] -= quantity

                capacity -= quantity * production_times[second_product]

                # Produce the first product if there is spare capacity
                if capacity > 0 and solution_full[period, Skip_production] == 0:
                    max_quantity = capacity // production_times[first_product]
                    quantity = min(max_quantity, backlog[period, first_product])
                    decoded[period, 1] = quantity
                    for j in range(periods):
                        backlog[j, first_product] -= quantity

            # If we do not change the setup, just produce the second product 
            else:
                max_quantity = capacity // production_times[second_product]
                quantity = min(max_quantity, backlog[period, second_product])
                decoded[period + 1, 0] = quantity
                for j in range(periods):
                    backlog[j, second_product] -= quantity

                decoded[period, 2] = second_product

        elif solution_full[period, setup_change_direct] == 1:

            if backlog[period, decoded[period+1, 2]] > 0:
                second_product = decoded[period+1, 2]
                decoded[period, 2] = second_product
                spare_capacity = capacities[period] - setup_times[second_product, decoded[period + 1, 2]]
                max_quantity = spare_capacity // production_times[second_product]
                quantity = min(max_quantity, backlog[period, second_product])
                decoded[period, 1] = quantity
                for j in range(periods):
                    backlog[j, second_product] -= quantity

            elif solution_full[period, Skip_production] == 0:
                decoded[period, 2] = second_product
                spare_capacity = capacities[period] - setup_times[second_product, decoded[period + 1, 2]]
                max_quantity = spare_capacity // production_times[second_product]
                quantity = min(max_quantity, backlog[period, second_product])
                decoded[period, 1] = quantity
                for j in range(periods):
                    backlog[j, second_product] -= quantity

            else:
                decoded[period, 2] = decoded[period + 1, 2]


        # If the product with the highest priority differs from the one in the last period, change the setup directly
        elif solution_full[period, Skip_production] == 0:
            decoded[period, 2] = second_product
            spare_capacity = capacities[period] - setup_times[second_product, decoded[period + 1, 2]]
            max_quantity = spare_capacity // production_times[second_product]
            quantity = min(max_quantity, backlog[period, second_product])
            decoded[period, 1] = quantity
            for j in range(periods):
                backlog[j, second_product] -= quantity

        # If we skip production, just transfer the setup from the last period
        else:
            decoded[period, 2] = decoded[period + 1, 2]

    # If the backlog matrix is > 0, the solution is infeasible and must be punished
    shortage = np.sum(backlog[0]) * penalty_shortage

    # ========== CALCULATE FITNESS ==========
    lots1 = decoded[1:, 0]
    lots2 = decoded[:-1, 1]
    seq = decoded[:, 2]

    # Create a matrix with the production quantites (Periods x Products)
    production = np.zeros((periods, products), dtype=np.int32)
    for t in range(periods):
        production[t, seq[t + 1]] += lots1[t]
        production[t, seq[t]] += lots2[t]

    # Add the production quantities up to get the inventory matrix (Periods x Products)
    inventory = np.zeros((periods, products), dtype=np.int32)
    for m in range(products):
        cum = 0
        for t in range(periods):
            cum += production[t, m] - demand[t, m]
            inventory[t, m] = cum

    # Calaculate holding and shortage costs
    hold_cost = 0
    for t in range(periods):
        for m in range(products):
            inv = inventory[t, m]
            hold_cost += max(inv,0) * holding_costs[m]

    #print(inventory)

    setup_cost = 0
    for t in range(periods):
        setup_cost += setup_costs[seq[t], seq[t + 1]]

    total_cost = hold_cost + shortage + setup_cost

    return production, decoded, total_cost



In [20]:
problem = problem_excel_x2

In [21]:
result,algorithm = main(problem,100,300,num_children=100,
           stagnation_limit=1000,num_subpopulations=48)

Compiling Functions ...


Iteration: 4526, Best: 12586: : 91.0it [01:30,  1.01it/s]                      


=== Production Matrix ===
[[ 0  0  0  0  0 28  0  0]
 [ 0  0  0  0  0 22  1  0]
 [ 0  0  0  0  0  0 38  0]
 [ 0  0  0 19  0  0  9  0]
 [ 0  0  0 20  0  0  0  0]
 [ 0  0  0  0 26  0  0  0]
 [ 0  0  0  0 15  0  0  0]
 [ 0  0  0  0 38  0  0  0]
 [ 0  0  0  0 15  3  0  0]
 [ 0  0  0  0  0 38  0  0]
 [ 0  0  0  0  0  1  0 25]
 [ 0  0  3  0  0  0  0 24]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0 19  0  0  0  0  0]
 [ 0 12 11  0  0  0  0  0]
 [ 0 38  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0 33  0  0  0  0  0  0]
 [ 0 38  0  0  0  0  0  0]
 [ 0  5  0  0  0 19  0  0]
 [ 0  0  0  2  0 20  0  0]
 [ 0  0  0 38  0  0  0  0]
 [ 0  0  0 31  0  0  0  0]
 [ 0  0  4 18  0  0  0  0]
 [ 0  0 38  0  0  0  0  0]
 [ 0  0  0  0  0 26  0  0]
 [ 0  0  0  0  0 14  9  0]
 [ 0  0  0  0  0  0 38  0]
 [ 0  0  0  0  0  0 38  0]
 [ 0  0  0  0  0  0 23  0]
 [ 0  0  0  0  0 31  0  0]
 [ 0  0  0  0  0 38  0  0]
 [ 0  0  0  0  5 13  0  0]
 [ 0  0  0  0 38  0  0  0]
 

In [22]:
sol = np.random.rand(60,8)
sol2 = np.random.randint(2,size=(60,3))
result = np.hstack([sol,sol2])

In [28]:
  
sol_prod, sol_dec, sol_fit = decode_full(problem.demand,
                                                    problem.capacities,
                                                    problem.production_times,
                                                    problem.setup_times,
                                                    result,
                                                    problem.holding_costs,
                                                    problem.setup_costs,
                                                    1000)
tl = run_tabu_search(problem,result,sol_dec,sol_fit,5000,30,200,8)

Iteration: 10286, Best: 13840, Current: 17193: : 10285it [00:30, 342.79it/s]                


=== Production Matrix ===
[[ 0  0  0  0  0 28  0  0]
 [ 0  0  0  0  0 22  1  0]
 [ 0  0  0  0  0  0 38  0]
 [ 0  0  0 19  0  0  9  0]
 [ 0  0  0 20  0  0  0  0]
 [ 0  0  0  0 26  0  0  0]
 [ 0  0  0  0  1  4  0  0]
 [ 0  0  0  0  0 38  0  0]
 [ 0  0  0  0 18  0  0  0]
 [ 0  0  0  0 26  0  0  0]
 [ 0  0  3  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 29]
 [ 0  0  0  0  0  0  0 20]
 [ 0  0  7  0  0  0  0  0]
 [ 0  0 23  0  0  0  0  0]
 [ 0 29  0  0  0  0  0  0]
 [ 0 21  0  0  0  0  0  0]
 [ 0  0  0  0 23  0  0  0]
 [ 0  0  0  0 37  0  0  0]
 [ 0  0  0  0  4 14  0  0]
 [ 0  0  0  0  0 22  0  0]
 [ 0  0  0 27  0  0  0  0]
 [ 0  0  0 25  0  0  0  0]
 [ 0 38  0  0  0  0  0  0]
 [ 0 38  0  0  0  0  0  0]
 [ 0  0 23  0  0  0  0  0]
 [ 0  0 16  0  0 10  0  0]
 [ 0  0  0  0  0 38  0  0]
 [ 0  0  0  0  0 21  2  0]
 [ 0  0  0  0  0  0 28  0]
 [ 0  0  0 38  0  0  0  0]
 [ 0  0  0 38  0  0  0  0]
 [ 0  0  0 11  0  0 17  0]
 [ 0  0  0  0  0  0 38  0]
 [ 0  0  0  0  0  0 23  0]
 [ 0  0  0  0  0 38  0  0]
 

In [31]:
tl.queue

deque([-395122968962411036,
       7458395095490720152,
       491404084391903455,
       -4320906465804473172,
       802282920845288073,
       3470448136078486233,
       6740752462530750865,
       -5730640080498610097,
       -4168620678620562860,
       -1567260983131584082,
       7813549339125690489,
       3372781098165249698,
       7809413333253784908,
       -780151400013360863,
       -3330097551751833408,
       170524618251744862,
       1782274551116998482,
       2254068858759538825,
       -3992051769670798862,
       4997911667682266017,
       6631236838876169534,
       -5712618555997883721,
       6769512092956647233,
       -1636955454711064551,
       -8495163222524613424,
       -2880682763195377709,
       -2852053065998030976,
       5913393485058014033,
       6237874413181790748,
       5841045676538415986,
       989756676512342296,
       -7805578016891142526,
       -4463411772652893544,
       -7731573974739807910,
       -4211399976216936315,
       -3